# Predict low high models
Model selection for predict lo hi


To install conda and binance packages to this notebook uncomment the code below

In [1]:
#%conda install -c plotly plotly=5.9.0
#%conda install pip
#%conda install twisted
%pip install plotly==5.9.0
%pip install twisted
#%pip install binance-connector==1.13.0
%pip install pandas
%pip install scikit-learn
%pip install tensorflow==2.11.0
%pip install keras==2.11.0
%pip install scikeras


import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable


Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Feature engineering
Read data from bot's history, resample to equal intervals, create X, y

In [2]:
import os
import glob
import pandas as pd
from datetime import timedelta,date

def read_last_data(strategy, days=1):
    """ Read last last day from bot history """
    
    def read_last(symbol: str, name: str, days=1):
        data_dir=f"./../data/yandex-cloud/{strategy}/Xy"
        file_paths = sorted([f"{data_dir}/{f}" for f in os.listdir(data_dir) if f.endswith(f"{symbol}_{name}.csv")])[-days:]
        return pd.concat([pd.read_csv(f, parse_dates=True, index_col="datetime") for f in file_paths])

    data=read_last("BTCUSDT", "data", days)
    X=read_last("BTCUSDT", "X", days)
    y=read_last("BTCUSDT", "y", days)
    return data,X,y

#strategy="SimpleKerasStrategy"
strategy="LSTMStrategy"
bidask,X_bot,y_bot = read_last_data(strategy, days=1)
bidask

,symbol,bid,bid_vol,ask,ask_vol
datetime,,,,,
2023-05-12 08:59:39.963468,BTCUSDT,26290.98,3.16301,26290.99,6.23509
2023-05-12 08:59:41.871429,BTCUSDT,26290.98,2.97283,26290.99,6.23509
2023-05-12 08:59:42.107489,BTCUSDT,26290.98,2.97585,26290.99,6.23509
2023-05-12 08:59:42.447281,BTCUSDT,26290.98,2.97585,26290.99,6.42527
2023-05-12 08:59:42.647319,BTCUSDT,26290.98,2.94358,26290.99,6.42527
...,...,...,...,...,...
2023-05-12 09:42:58.178472,BTCUSDT,26344.78,5.83724,26344.79,1.64233
2023-05-12 09:43:16.740793,BTCUSDT,26344.78,5.83724,26344.79,1.64146
2023-05-12 09:43:25.724070,BTCUSDT,26344.78,5.85666,26344.79,1.64146


Resample data to use equal time intervals

In [3]:
import re
def get_targets(bidask, predict_window="10s"):
    """ 
    Calculate targets - bid/ask bounds in future prediction window
    """
    fut_min=bidask[["bid", "ask"]][::-1].rolling(predict_window).min()[::-1].rename(columns={"bid":"bid_min_fut", "ask":"ask_min_fut"})
    fut_max=bidask[["bid", "ask"]][::-1].rolling(predict_window).max()[::-1].rename(columns={"bid":"bid_max_fut", "ask":"ask_max_fut"})
    return pd.concat([fut_min, fut_max], axis = 1)
    
def resampled(X: pd.DataFrame, y: pd.DataFrame, time_interval) -> (pd.DataFrame, pd.DataFrame):
    """ Resample to make equal intervals time series """
    if int(re.sub(r'\D', "", time_interval)) == 0:
        return X, y
    Xy=pd.concat([X,y], axis=1)
    diffmap = [(c, "sum") for c in Xy.columns if c.endswith("diff")]
    timemap = [(c, "last") for c in Xy.columns if c.startswith("time") and not c.endswith("diff")]
    l2map = [(c, "last") for c in Xy.columns if c.startswith("l2_")]
    
    futmap = [(c, "last") for c in Xy.columns if c.endswith("_fut")]
    colmap = dict(diffmap + timemap + l2map + futmap + [("spread", "last")])
    resampled = Xy.resample(time_interval).agg(colmap).dropna()
    return resampled[X.columns], resampled[y.columns]

predict_window="10s"
resample_interval="0s"
y_raw = get_targets(bidask, predict_window)
X,y = resampled(X_bot, y_raw, resample_interval)
#X,y=X_bot,y_raw
pd.concat([X,y], axis=1).dropna()

,time_hour,time_minute,time_second,time_day_of_week,time_diff,spread,bid_diff,bid_vol_diff,ask_diff,ask_vol_diff,l2_bucket_-10,l2_bucket_-9,l2_bucket_-8,l2_bucket_-7,l2_bucket_-6,l2_bucket_-5,l2_bucket_-4,l2_bucket_-3,l2_bucket_-2,l2_bucket_-1,l2_bucket_0,l2_bucket_1,l2_bucket_2,l2_bucket_3,l2_bucket_4,l2_bucket_5,l2_bucket_6,l2_bucket_7,l2_bucket_8,l2_bucket_9,bid_min_fut,ask_min_fut,bid_max_fut,ask_max_fut
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-05-12 08:59:39.963468,8,59,39,4,3.935130e+08,0.01,0.0,-0.01603,0.0,0.00000,0.00091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,9.27933,14.47269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26290.98,26290.99,26290.98,26290.99
2023-05-12 08:59:41.871429,8,59,41,4,1.907961e+09,0.01,0.0,-0.19018,0.0,0.00000,0.00091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,9.27933,14.47269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26290.98,26290.99,26290.98,26290.99
2023-05-12 08:59:42.107489,8,59,42,4,2.360600e+08,0.01,0.0,0.00302,0.0,0.00000,0.00091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,9.27933,14.47269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26290.98,26290.99,26290.98,26290.99
2023-05-12 08:59:42.447281,8,59,42,4,3.397920e+08,0.01,0.0,0.00000,0.0,0.19018,0.00091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,9.27933,14.47269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26290.98,26290.99,26290.98,26290.99
2023-05-12 08:59:42.647319,8,59,42,4,2.000380e+08,0.01,0.0,-0.03227,0.0,0.00000,0.00091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,9.27933,14.47269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26290.98,26290.99,26290.98,26290.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-12 09:42:58.178472,9,42,58,4,1.071369e+10,0.01,0.0,-0.42000,0.0,0.00000,0.00091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03877,288.49965,15.90145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26344.78,26344.79,26344.78,26344.79
2023-05-12 09:43:16.740793,9,43,16,4,1.856232e+10,0.01,0.0,0.00000,0.0,-0.00087,0.00091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03877,288.49965,15.90145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26344.78,26344.79,26344.78,26344.79
2023-05-12 09:43:25.724070,9,43,25,4,8.983277e+09,0.01,0.0,0.01942,0.0,0.00000,0.00091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03877,288.49965,15.90145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26344.78,26344.79,26344.78,26344.79


## Create the model


In [16]:
from keras import Sequential, Input
from keras.layers import Dense, Dropout
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from scikeras.wrappers import KerasRegressor
from keras.layers import *
from keras.layers import LSTM
from sklearn.compose import ColumnTransformer, make_column_selector
import matplotlib.pyplot as plt
import tensorflow as tf


  
def create_model(X_train, y_train, window_size):
    model = Sequential()
    model.add(LSTM(128,  return_sequences=True, input_shape=(window_size, X_train.shape[1])))

#     model.add(ConvLSTM2D(filters=64, 
#                                 kernel_size=(1, 1),
#                                 padding='same', 
#                                 return_sequences=True,
#                                 activation="tanh",
#                                 ))
    model.add(MaxPooling(pool_size=(2,2)))    
    model.add(Dropout(0.2))
    model.add(LSTM(32))         
    model.add(Dropout(0.2))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='linear'))
    #model.add(Dense(y_train.shape[1], activation='softmax'))
    model.compile(optimizer='adam', loss='mae', metrics=['mse'])
    return model

def create_model_prod(X_train, y_train, window_size):
    model = Sequential()
    model.add(LSTM(128,  return_sequences=True, input_shape=(window_size, X_train.shape[1])))
    model.add(Dropout(0.2))
    model.add(LSTM(32))         
    model.add(Dropout(0.2))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='linear'))
    #model.add(Dense(y_train.shape[1], activation='softmax'))
    model.compile(optimizer='adam', loss='mae', metrics=['mse'])
    return model


def plot_history(history):
    """ Plot history loss and metrics"""
    # summarize history for accuracy
    for metric_name in history.history:
        # validation metrics names will be calculated from related train metrics
        if metric_name.startswith("val_"): continue

        # Plot metric and related test (val_..) metric
        names=[metric_name, f"val_{metric_name}"]
        for name in names:
            plt.plot(history.history[name])
        # Captions and show the plot
        plt.title(metric_name)
        plt.ylabel(metric_name)
        plt.xlabel('epoch')
        plt.legend(names, loc='upper left')
        plt.show()

test_size=0.3
test_index = int(len(X)*(1-test_size))
X_train, y_train, X_test, y_test = X.iloc[:test_index], y.iloc[:test_index], X.iloc[test_index:], y.iloc[test_index:]


# Train/test split    
x_pipe = Pipeline(
    [("xrs", RobustScaler()),
     #("xss", StandardScaler()),     
     ("xmms", MinMaxScaler())])
x_pipe.fit(X_train)

y_pipe = Pipeline(
    [("yrs", RobustScaler()),
     #("yss",StandardScaler()),     
     ("ymms",MinMaxScaler())])
y_pipe.fit(y_train)

#x_scaler, y_scaler = MinMaxScaler().fit(X_train), MinMaxScaler().fit(y_train)
X_train, y_train = x_pipe.transform(X_train), y_pipe.transform(y_train)
X_test, y_test = x_pipe.transform(X_test), y_pipe.transform(y_test)
print(f"Train size: {X_train.shape[0]}")


# Create the model
window_size = 10
model = create_model(X_train, y_train, window_size)

# Fit the model
epochs=50
steps_per_epoch=5
train_gen = TimeseriesGenerator(X_train, y_train, length=window_size)
test_gen = TimeseriesGenerator(X_test, y_test, length=window_size)
history=model.fit(train_gen, validation_data=test_gen, epochs=epochs, steps_per_epoch=steps_per_epoch)

# Plot the result
plot_history(history)


Train size: 8971


NameError: name 'MaxPooling' is not defined